# Installing OpenVINO on Colab und running demo

In [ ]:
!apt-get update && apt-get upgrade -y &&\
    apt-get autoremove -y
!sudo apt install -y --no-install-recommends \
      build-essential \
      cpio \
      curl \
      git \
      lsb-release \
      pciutils \
      python3.6 \
      python3-pip \
      sudo \
      libusb-1.0-0 libboost-program-options1.62.0 \
      libboost-thread1.62.0 libboost-filesystem1.62.0 \
      libssl1.0.0 libudev1 libjson-c3 usbutils udev wget

In [ ]:
!OV_LINK=https://registrationcenter-download.intel.com/akdlm/irc_nas/17504/l_openvino_toolkit_p_2021.2.185.tgz &&\
 curl $OV_LINK --output l_openvino_toolkit.tgz
openvino_path = "l_openvino_toolkit.tgz"
  
!tar xf "{openvino_path}" &&\
    cd l_openvino_toolkit_p* && \
    ./install_openvino_dependencies.sh &&\
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg  


In [ ]:
!source /opt/intel/openvino_2021.2.185/bin/setupvars.sh &&\
 /opt/intel/openvino_2021.2.185/deployment_tools/demo/demo_squeezenet_download_convert_run.sh

# Optimizing Tensorflow-Model yolo v3


In [ ]:
 !pip install imgaug==0.2.5
 !pip install gast==0.3.2
 !pip install folium==0.2.1

Next cell gives an error, but if it's only one about version match - ignore it

In [ ]:
!/opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/install_prerequisites/install_prerequisites_tf.sh*

Next cell should be ran for creating folders for models for 
simplicity

In [17]:
mkdir ~/downloaded_models ~/optimized_models ~/samples

Now we can download already pretrained models from OpenVINO source

In [ ]:
!python3 /opt/intel/openvino_2021.2.185/deployment_tools/tools/model_downloader/downloader.py --name yolo-v3-tf --output_dir /root/downloaded_models/

In official [documentation](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_tf_specific_Convert_YOLO_From_Tensorflow.html) the general case is described. But our problem can be solved according [that](https://github.com/openvinotoolkit/openvino/issues/1593) answer: we use mo.py, and on [github](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/yolo-v3-tf/model.yml) (officially) there are flags for script **only for open zoo pretrained models**
[Текст ссылки]

In [ ]:
!source /opt/intel/openvino_2021.2.185/bin/setupvars.sh &&\
 python3 /opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/mo.py \
  --framework=tf \
  --output_dir=/root/optimized_models \
  --model_name=yolov3_25jan_3 \
  --data_type=FP16 \
  --input_shape=[1,416,416,3] \
  --input=input_1 \
  --scale_values=input_1[255] \
  --reverse_input_channels \
  --transformations_config=/root/downloaded_models/public/yolo-v3-tf/yolo-v3.json \
  --input_model=/root/downloaded_models/public/yolo-v3-tf/yolo-v3.pb

In [ ]:
#!python3 /opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/mo_tf.py --input_model /root/downloaded_models/public/yolo-v3-tf/public/yolo-v3-tf/yolo-v3.pb --input_shape [1,416,416,3] --output_dir ~/optimized_models/

In [ ]:
#!source /opt/intel/openvino_2021.2.185/bin/setupvars.sh

In [ ]:
#!python3 /opt/intel/openvino_2021.2.185/deployment_tools/open_model_zoo/demos/python_demos/object_detection_demo/object_detection_demo.py --model /root/optimized_models/yolo-v3.xml -at={yolo} --input /opt/intel/openvino_2021.2.185/deployment_tools/demo/car.png -d CPU 

# Optimizing Tensorflow-Model yolo-v3-tiny

If you made next steps for yolo-v3 (not tiny),  just ignore it till "Dowloading..."

In [ ]:
 !pip install imgaug==0.2.5
 !pip install gast==0.3.2
 !pip install folium==0.2.1

Next cell gives an error, but if it's only one about version match - ignore it

In [ ]:
!/opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/install_prerequisites/install_prerequisites_tf.sh*

Next cell should be ran for creating folders for models for 
simplicity

In [ ]:
mkdir ~/downloaded_models ~/optimized_models ~/samples

Downloading. Now we can download already pretrained models from OpenVINO source

In [ ]:
!python3 /opt/intel/openvino_2021.2.185/deployment_tools/tools/model_downloader/downloader.py --name yolo-v3-tiny-tf --output_dir /root/downloaded_models/

In official [documentation](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_tf_specific_Convert_YOLO_From_Tensorflow.html) the general case is described. But our problem can be solved according [that](https://github.com/openvinotoolkit/openvino/issues/1593) answer: we use mo.py, and on [github](https://github.com/openvinotoolkit/open_model_zoo/blob/master/models/public/yolo-v3-tiny-tf/model.yml)  (officially) there are flags for script **only for open zoo pretrained models**

In [ ]:
!source /opt/intel/openvino_2021.2.185/bin/setupvars.sh &&\
 python3 /opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/mo.py \
  --framework=tf \
  --output_dir=/root/optimized_models \
  --model_name=yolov3tiny_25jan_3 \
  --input_shape=[1,416,416,3] \
  --data_type=FP16 \
  --input=image_input \
  --scale_values=image_input[255] \
  --reverse_input_channels \
  --transformations_config=/root/downloaded_models/public/yolo-v3-tiny-tf/yolo-v3-tiny-tf/yolo-v3-tiny-tf.json \
  --input_model=/root/downloaded_models/public/yolo-v3-tiny-tf/yolo-v3-tiny-tf/yolo-v3-tiny-tf.pb

# Run inference with object_detection_demo.py

When running object_detection_demo.py OpenVINO enviroment should be initialized in one cell with demo running code. The flags "-r" means that output will be in raw data format and "no_show" will not call the OpenCV's imshow, that works incorrectly in Colab. I run every optimized model just to see, that output raw data has no over1000 bboxes - that means, that the model was optimized incorrectly (in most cases) or was wrong learned

In [ ]:
!source /opt/intel/openvino_2021.2.185/bin/setupvars.sh &&\
 python3 /opt/intel/openvino_2021.2.185/deployment_tools/open_model_zoo/demos/python_demos/object_detection_demo/object_detection_demo.py \
 --model /root/optimized_models/yolov3tiny_25jan_3.xml\
 -at={'yolo'} \
 --input /opt/intel/openvino_2021.2.185/deployment_tools/demo/car.png \
 -d CPU \
 -r \
 --no_show

In [ ]:
#from google.colab.patches import cv2_imshow
#!apt-get install gtk

In [ ]:
#!find /opt/intel -name *.png

# Upload Intermediate Representation to Google Drive

In [24]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


Goog [guide](https://colab.research.google.com/drive/1CN7JxLA_Qk1Clc4CM75rSAqIlUKIF8xZ#scrollTo=T_tDc-oMEY22) for model saving

And [there](https://github.com/googlecolab/colabtools/issues/287) is described, why you can't see already uploaded to /content/drive folder in Google Drive instantly

In [ ]:
mkdir /content/gdrive/MyDrive/yolo_v3_tiny_from_colab

In [39]:
!cp /root/optimized_models/yolov3tiny_25jan_3.xml  /content/gdrive/MyDrive/yolo_v3_tiny_from_colab/
!cp /root/optimized_models/yolov3tiny_25jan_3.bin  /content/gdrive/MyDrive/yolo_v3_tiny_from_colab/ 

In [40]:
!cp /root/optimized_models/yolov3_25jan_3.xml  /content/gdrive/MyDrive/yolo_v3_from_colab/
!cp /root/optimized_models/yolov3_25jan_3.bin  /content/gdrive/MyDrive/yolo_v3_from_colab/ 

Sync can be manually with following comand (with small-sized files not neccessary)

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

# Download files from mounted Colab to the local machine

In [ ]:
from google.colab import files
files.download( "/root/optimized_models/yolo-v3.bin" ) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download( "/root/optimized_models/yolo-v3.xml" )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>